In [ ]:
import numpy as np 
import pandas as pd
import os

print(os.listdir("../input"))
print(os.listdir("../working"))

import warnings
warnings.filterwarnings('ignore')
from fastai import *
from fastai.vision import *
import matplotlib
import re
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
train_data = pd.read_csv('../input/train.csv')
test_data = pd.read_csv('../input/test.csv')
print(train_data.shape)
print(test_data.shape)

In [ ]:
input_training_images = train_data.iloc[:,1:785].values
training_images = np.array([np.reshape(i, (28, 28)) for i in input_training_images])
training_labels = train_data.label

input_testing_images = test_data.iloc[:,0:785].values
testing_images = np.array([np.reshape(i, (28, 28)) for i in input_testing_images])

print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)

In [ ]:
train_count = -1
test_count = -1
os.mkdir("../working/images/")
os.mkdir("../working/images/test/")
os.mkdir("../working/images/train/")

#saving train files with naming convention: aaaa_b.jpeg (aaaa - counter, b - label). Eg. 15443_5.jpeg

for i in range(len(training_labels)):
    train_count +=1
    train_files = matplotlib.image.imsave('../working/images/train/' + str("%04d" % train_count) + '_' + str(training_labels.iloc[i]) + '.jpeg',
                                          training_images[training_labels.index[i]], cmap='gray')
    
#saving test files with naming convention: aaaa.jpeg (aaaa - counter). Eg. 15443.jpeg
for j in range(len(testing_images)):
    test_count +=1
    test_files = matplotlib.image.imsave('../working/images/test/' + str("%04d" % test_count) + '.jpeg',
                                          testing_images[j], cmap='gray')


In [ ]:
print(len(os.listdir("../working/images/train")))
print(len(os.listdir("../working/images/test")))

In [ ]:
train_path_img = "../working/images/train"
train_fnames = get_image_files("../working/images/train/")
train_fnames[:5]

In [ ]:
test_path_img = "../working/images/test"
test_fnames = get_image_files("../working/images/test/")
test_fnames[:5]

In [ ]:
np.random.seed(2)
pat = re.compile(r'([0-9])\.jpeg$')

In [ ]:
data = ImageDataBunch.from_name_re(train_path_img,
                                   train_fnames,
                                   pat,
                                   test=test_path_img,
                                   ds_tfms=get_transforms(do_flip=False),
                                   size=28, bs=64, num_workers=0).normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=3, figsize=(5,5))

In [ ]:
print(data.classes)
len(data.classes),data.c

In [ ]:
learn = create_cnn(data, models.resnet34, metrics=error_rate)

In [ ]:
learn.fit_one_cycle(4)